# First Things First...

Importing and installing required libraries. This might take a while to run.

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

!apt update -qq 2>/dev/null >/dev/null
!apt-get install -y libvips libvips-dev libvips-tools -qq 2>/dev/null >/dev/null

%pip install pytorch_lightning --quiet
%pip install pyvips --quiet

import os
import sys
from os.path import join

import torch
import pytorch_lightning as pl

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

Defining the main project path and adding it to path. This is necessary since we will import a lot functions from different directories in this path.

In [ ]:
PROJ_PATH = "drive/MyDrive/vae"
sys.path.insert(0, PROJ_PATH)

## Ensuring that all operations are deterministic on GPU for reproducibility.

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

**IMPORTANT:** Make sure to change the following made-up student ID with your real UTA  ID.

In [ ]:
STUDENT_ID = 1001234567
pl.seed_everything(STUDENT_ID)

# Downloading your unique dataset.

In [ ]:
from utils.download_tools import download_gdc_data

In [ ]:
DATA_PATH = join(PROJ_PATH, "data")
GDC_CSV_PATH = join(DATA_PATH, "gdc_breast.csv")

In [ ]:
download_gdc_data(STUDENT_ID, num_samples=3, csv_path=GDC_CSV_PATH, save_dir=DATA_PATH)

# Training

In [ ]:
GENERAL_DIR = join(PROJ_PATH, "general")
LOGGING_DIR = join(GENERAL_DIR, "logs")
LOGGING_NAME = 'trial'

In [ ]:
%tensorboard --logdir {LOGGING_DIR}/{LOGGING_NAME}

In [ ]:
torch.cuda.empty_cache()

%run {GENERAL_DIR}/train.py \
    --STUDENT_ID {STUDENT_ID}\
    --everything_seed {STUDENT_ID}\
    --logging_dir {LOGGING_DIR}\
    --logging_name {LOGGING_NAME}\
    --data_dir {DATA_PATH}\
    --patch_size 512\
    --n_patches_per_image 100\
    --whitespace_threshold 0.82\
    --num_dataset_workers 4\
    --split_ratio 0.8 0.1 0.1\
    --normalize_transform\
    --num_dataloader_workers 4\
    --batch_size 4\
    --input_height 512\
    --enc_type "resnet18"\
    --enc_out_dim 512\
    --kl_coeff 0.1\
    --latent_dim 64\
    --lr 5e-4\
    --accelerator "gpu"\
    --strategy "auto"\
    --devices 1\
    --max_epochs 20
    # --coords_already_generated